<h1 align="center">Monte Carlo Tree Search</h1>

## 1. Mô tả bài toán

Có 4 nút: [0, 1, 2, 3].

2 người chơi luân phiên chọn một nút chưa được chọn.

Ai chọn được 2 nút liền nhau (như 1 và 2) thì thắng.

Nếu chọn hết mà không ai thắng → hòa.

## 2. Dữ liệu trạng thái

In [34]:
def create_state():
    return {
        "player_moves": [set(), set()],   # danh sách các nút mà mỗi người chơi đã chọn
        "available_moves": [0, 1, 2, 3],  # các nước đi còn lại
        "current_player": 0               # 0 hoặc 1
    }


## 3. Hàm kiểm tra thắng thua

In [36]:
def check_winner(player_moves):
    for i in range(3):
        if i in player_moves and (i + 1) in player_moves:
            return True
    return False


## 4. Hàm mô phỏng một ván ngẫu nhiên

In [16]:
import random

def simulate_random_playout(state):
    state = {
        "player_moves": [set(state["player_moves"][0]), set(state["player_moves"][1])],
        "available_moves": state["available_moves"][:],
        "current_player": state["current_player"]
    }

    while True:
        if not state["available_moves"]:
            return -1  # hòa

        move = random.choice(state["available_moves"])
        state["player_moves"][state["current_player"]].add(move)
        state["available_moves"].remove(move)

        if check_winner(state["player_moves"][state["current_player"]]):
            return state["current_player"]

        state["current_player"] = 1 - state["current_player"]


## 5. Thuật toán MCTS 

In [18]:
def mcts(state, simulations=1000):
    move_scores = {}
    for move in state["available_moves"]:
        wins = 0
        plays = 0
        for _ in range(simulations):
            # Clone state
            sim_state = {
                "player_moves": [set(state["player_moves"][0]), set(state["player_moves"][1])],
                "available_moves": state["available_moves"][:],
                "current_player": state["current_player"]
            }

            # Apply the move
            sim_state["player_moves"][sim_state["current_player"]].add(move)
            sim_state["available_moves"].remove(move)
            if check_winner(sim_state["player_moves"][sim_state["current_player"]]):
                winner = sim_state["current_player"]
            else:
                sim_state["current_player"] = 1 - sim_state["current_player"]
                winner = simulate_random_playout(sim_state)

            plays += 1
            if winner == state["current_player"]:
                wins += 1
            elif winner == -1:
                wins += 0.5  # hòa

        move_scores[move] = wins / plays

    # Chọn nước đi có tỷ lệ thắng cao nhất
    return max(move_scores, key=move_scores.get)


## 6. Chương trình chính

In [20]:
def play_game():
    state = create_state()

    while True:
        if not state["available_moves"]:
            print("Hòa!")
            return

        move = mcts(state, simulations=500)
        print(f"Người chơi {state['current_player']} chọn nút {move}")
        state["player_moves"][state["current_player"]].add(move)
        state["available_moves"].remove(move)

        if check_winner(state["player_moves"][state["current_player"]]):
            print(f"Người chơi {state['current_player']} thắng!")
            return

        state["current_player"] = 1 - state["current_player"]


## 7. Chạy thử chương trình

In [40]:
play_game()

Người chơi 0 chọn nút 2
Người chơi 1 chọn nút 1
Người chơi 0 chọn nút 3
Người chơi 0 thắng!
